In [1]:
import pandas as pd
import numpy as np
import random
import csv
import googlemaps
import sys
import math
from math import sin, cos, sqrt, atan2, radians, pi

from surprise import KNNWithMeans, SVD, KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

In [2]:
RATINGS_PATH = './dataset_test/rating_avaliacao.csv'
POIS_PATH = './dataset/pois.csv'
#POIS_PATH = './dataset_test/all_pois.csv'
USER_PATH = './dataset_test/user_avaliacao.csv'

In [9]:
class Recommendations:
    
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    
    def __init__(self, user_data='', rating_data='', data_frame='', poi_data=''):
        
        if rating_data:
            ''' userId,poiId,rating '''
            reader = Reader(sep=',')
            self.ratings = Dataset.load_from_file(rating_data, reader)
            self.trainset = self.ratings.build_full_trainset()
            self.sim_options = {'name': 'cosine','user_based': False}
            #self.df_ratings = pd.read_csv(rating_data, low_memory=False, names=['userId','latitude','longitude','poiId','rating'])
            self.df_ratings = pd.read_csv(rating_data, low_memory=False, names=['userId','poiId','rating'])
        elif not data_frame.empty:
            reader = Reader(rating_scale=(0, 5))
            self.ratings = Dataset.load_from_df(data_frame[['userId', 'poiId', 'rating']], reader)
            self.trainset = self.ratings.build_full_trainset()
            self.sim_options = {'name': 'cosine','user_based': False}
        if poi_data:
            ''' poiId,latitude,longitude,name,preferenceid,preference '''
            self.pois = pd.read_csv(poi_data, low_memory=False)
            self.pois = pd.DataFrame(self.pois, columns=['poiId','latitude','longitude','name','category','address'])
            #self.csv_reader = csv.reader(self.pois, delimiter=',') 
            print(self.pois)
        if user_data:
            ''' userId,name,latitude,longitude,id_preferencia,preference '''
            self.users = pd.read_csv(user_data, low_memory=False)
            #self.csv_reader = csv.reader(pois, delimiter=',') 

In [10]:
recommendations = Recommendations(user_data=USER_PATH, rating_data=RATINGS_PATH, poi_data=POIS_PATH)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   poiId      406 non-null    int64  
 1   latitude   406 non-null    float64
 2   longitude  406 non-null    float64
 3   name       406 non-null    object 
 4   category   406 non-null    object 
 5   address    406 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 19.2+ KB
None
     poiId   latitude  longitude  \
0        1 -12.936688 -38.394585   
1        2 -12.937388 -38.395224   
2        3 -12.909218 -38.351816   
3        4 -12.909454 -38.350678   
4        5 -12.935377 -38.392347   
5        6 -12.913722 -38.335701   
6        7 -12.913740 -38.334495   
7        8 -12.981759 -38.464242   
8        9 -13.012573 -38.483431   
9       10 -12.939290 -38.342214   
10      11 -12.978239 -38.455099   
11      12 -12.909511 -38.350743   
12      13 -12.981706 -38.464626   
13  